<a href="https://colab.research.google.com/github/Devarsh-19/Vegetable-classification/blob/main/project_veg_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!pip install -U matplotlib

     |████████████████████████████████| 11.2 MB 9.1 MB/s 
     |████████████████████████████████| 930 kB 56.5 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
import os # file directory and file sysstem ops
import numpy as np # for nd arrays

import torch  #for deep learning
from torch import nn

from PIL import Image
import cv2
from matplotlib import pyplot as plt

In [3]:
import torchvision
from torchvision import transforms, datasets

In [4]:
from tqdm import tqdm #progess report and loop opertarions

In [5]:
!mkdir ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle datasets download misrakahmed/vegetable-image-dataset

 98% 521M/534M [00:03<00:00, 186MB/s]
100% 534M/534M [00:03<00:00, 173MB/s]


In [8]:
!unzip -qq vegetable-image-dataset.zip

In [11]:
! mv Vegtable\ Images Vegetable Images 

mv: target 'Images' is not a directory


#EDA
--------------------
STEP 1 : Find number of classes and number of images in each class  
STEP 2 : Size Analysis - heights, widths, resolutions, aspect rations  
STEP 3 : Look for blurred images  
STEP 4 : Look for low contrast images  
STEP 5 : Look for noisy images  
STEP 6 : Look for bright and dark images  


In [12]:
train_vegetables = datasets.ImageFolder("Vegetable Images/train",transform = None)
val_vegetables = datasets.ImageFolder("Vegetable Images/validation",transform = None)
test_vegetables = datasets.ImageFolder("Vegetable Images/test",transform = None)

In [13]:
print(train_vegetables), print(val_vegetables), print(test_vegetables)

Dataset ImageFolder
    Number of datapoints: 15000
    Root location: Vegetable Images/train
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/validation
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/test


(None, None, None)

In [14]:
vegetable_classes = train_vegetables.classes
print(f"Classes: {vegetable_classes}")

Classes: ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']


In [15]:
print("No. of classes: {}".format(len(vegetable_classes)))

No. of classes: 15


In [16]:
targets = train_vegetables.targets

In [17]:
class_strength = np.unique(targets, return_counts= True)

In [18]:
class_strength

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000]))

EDA: STEP 2

In [19]:
widths = []
heights = []
resolutions = []
aspect_ratios = []
imgs = []

for i in tqdm(range(len(train_vegetables.imgs))):
  img_path, label = train_vegetables.imgs[i]
  img = Image.open(img_path)
  w, h = img.size

  imgs.append(img_path)
  heights.append(h)
  widths.append(w)
  resolutions.append(h*w)
  aspect_ratios.append(w/h)

100%|██████████| 15000/15000 [00:03<00:00, 3794.71it/s]


In [20]:
print(len(heights))
print(min(heights), max(heights))

15000
193 224


The standard resolution used to train the neural network images is 128 by 128.   
for 1st image 128 x 128  
for 2nd 224 x 224  
for 3rd 256 x 256 and so on  
make sure to keep resolution below 400 x 400  

IMAGE AUGMENTATION -> maipulate image data

In [21]:
transform = transforms.Compose([
                                transforms.Resize((224,224)),
                                transforms.ToTensor()
])

#on colab

train_vegetables = datasets.ImageFolder("Vegetable Images/train",transform = transform)
val_vegetables = datasets.ImageFolder("Vegetable Images/validation",transform = transform)
test_vegetables = datasets.ImageFolder("Vegetable Images/test",transform = transform)

In [22]:
print(train_vegetables), print(val_vegetables), print(test_vegetables)

Dataset ImageFolder
    Number of datapoints: 15000
    Root location: Vegetable Images/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/validation
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


(None, None, None)

### DATA SAMPLER AND RANDOM SAMPLING  
CUDA - Compute Unified Device Architecture
Validataion Data - Ground Truth

In [28]:
torch.device("cuda:0")

device(type='cuda', index=0)

In [24]:
train_loader = torch.utils.data.DataLoader(train_vegetables,
                                      batch_size = 32,
                                      shuffle = True,
                                      pin_memory = True)

In [25]:
test_loader = torch.utils.data.DataLoader(test_vegetables,
                                      batch_size = 45,
                                      shuffle = False,
                                      pin_memory = True)

In [26]:
val_loader = torch.utils.data.DataLoader(val_vegetables,
                                      batch_size = 40,
                                      shuffle = False,
                                      pin_memory = True)

In [27]:
print(len(train_loader)), print(len(test_loader)), print(len(val_loader))

469
67
75


(None, None, None)

In [30]:
#Create the sequential object

my_cnn_classifier = nn.Sequential()

In [32]:
#CREATING CNN LAYERS

#Add a CNN Set

#Convolution -> ReLU -> Pooling


conv_layer_1 = nn.Conv2d(in_channels= 3, out_channels= 32, kernel_size = 7, stride = 1, padding= 0, bias = True);

my_cnn_classifier.add_module("conv_layer_1", conv_layer_1)

act_1 = nn.ReLU();
my_cnn_classifier.add_module("act_1",act_1);

pool_1 = nn.AvgPool2d(kernel_size= 2, stride = 2,padding = 0);

my_cnn_classifier.add_module("pool_1",pool_1)

In [37]:
from torchsummary import summary
print(summary(my_cnn_classifier, (3,224,224), device = "cpu"))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 218, 218]           4,736
              ReLU-2         [-1, 32, 218, 218]               0
         AvgPool2d-3         [-1, 32, 109, 109]               0
Total params: 4,736
Trainable params: 4,736
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 26.11
Params size (MB): 0.02
Estimated Total Size (MB): 26.70
----------------------------------------------------------------
None
